In [1]:
from pathlib import Path
import pickle
from joblib import Parallel, delayed
from itertools import product
from smount_predictors.src.SeamountHelp import PipelinePredictor
from smount_predictors import SeamountHelp
import numpy as np
import pandas as pd
import simplekml
import xarray as xr
from interfaces_exclude.exclude_interface import exclude_interface

In [2]:
import os
os.chdir('data')

In [3]:
! gmt grdmath dist2coast.grd 20 GE = coast_mask.grd
! gmt grdmath depth_mask.grd coast_mask.grd MUL = swot_mask.grd
! gmt grdmath swot_mask.grd vgg_swot.grd MUL = swot_landmask.grd

In [4]:
os.chdir('..')

In [5]:
longitude_pairs = []
for lon in range(-180, 180, 5):
    longitude_pairs.append((lon, lon + 5))

latitude_pairs = []
for lat in range(-80, 80, 5):
    latitude_pairs.append((lat, lat + 5))

latlons = list(product(longitude_pairs, latitude_pairs))

In [6]:
model = pickle.load(open('out/cluster_tuned_model.pkl', 'rb'))
data_p = Path('data/swot_landmask.grd')

In [7]:
def recluster(model: PipelinePredictor, predictions: pd.DataFrame):
    def filter_clust_size(data: pd.DataFrame):
        def circle_ratio(data: pd.DataFrame):
            if abs(data['lon'].max() - data['lon'].min()) == 0:
                return 0
            if data.loc[:, 'cluster'].mean() == -1:
                return 1
            circle = abs(data['lat'].max() - data['lat'].min()) / abs(data['lon'].max() - data['lon'].min())
            mass = (abs(data['lat'].max() - data['lat'].min()) * abs(data['lon'].max() - data['lon'].min())) / data.shape[0]
            if circle == 0:
                return 0
            mass = 4 * mass / (circle * np.pi)  # Scale mass to be 1 if it is the correct mass of a circle
            return circle * mass
        circle_range = data
        divs = circle_range.groupby('cluster').apply(circle_ratio)
        divs = divs[(divs > np.mean(divs) - np.std(divs)) & (divs < np.mean(divs) + np.std(divs))]
        circle_range = circle_range.loc[(~circle_range['cluster'].isin(divs.index)) & (circle_range['cluster'] != -1)]
        return circle_range

    def norm_z(data: pd.DataFrame):
        data['norm_z'] = (data['z'] - data['z'].min()) / (data['z'].max() - data['z'].min())
        return data
    predictions = predictions.reset_index()
    clust_filt = predictions.copy()

    clust_filt = filter_clust_size(clust_filt)
    clust_filt = norm_z(clust_filt)
    recluster_index = (clust_filt['cluster'] != -1) & (clust_filt['norm_z'] > 0.7)
    clust_pred = clust_filt.loc[recluster_index]

    model.clusterer.fit_predict(clust_pred[['lon', 'lat']])
    clust_filt.loc[recluster_index, 'cluster'] = model.clusterer.labels_ + predictions['cluster'].max() + 1
    clust_filt.loc[~recluster_index, 'cluster'] = -1
    clust_filt.set_index(['lon', 'lat'], inplace=True)
    predictions.set_index(['lon', 'lat'], inplace=True)
    predictions.loc[clust_filt.index, 'cluster'] = clust_filt.loc[:, 'cluster']
    return predictions.reset_index()

In [8]:
def predict_zone(zone):
    lon = zone[0]
    lat = zone[1]
    data = SeamountHelp.readAndFilterGRD(data_p, lat, lon)
    data = exclude_interface(data, 'interfaces_exclude/vector_feats.xy', threshold=20)
    data = data.to_dataframe().reset_index()
    if np.all(data['z'] == 0):
        data['class'] = 0
        data['cluster'] = -1
        return data
    zone_pred = model.predict(data[['lon', 'lat', 'z']])
    return zone_pred

predictions = Parallel(n_jobs=-3)(delayed(predict_zone)(zone) for zone in latlons)
nulls = np.zeros(len(predictions), dtype=bool)
for idx, df in enumerate(predictions):  # ensure non-overlapping cluster numbers
    assert isinstance(df, pd.DataFrame)  # assertion for code linter typing features
    nulls[idx] = np.all(df['z'].values == 0)
    df.loc[df['cluster'] != -1, 'cluster'] = df.loc[df['cluster'] != -1, 'cluster'] + ((idx ** 2) * len(np.unique(df['cluster'])))
assert not np.all(nulls)
predictions = pd.concat(predictions)
predictions = recluster(model, predictions)
predictions['lon'] = np.degrees(predictions['lon'])
predictions['lat'] = np.degrees(predictions['lat'])
global_predictions = xr.Dataset.from_dataframe(predictions).set_coords(['lon', 'lat']).drop('index')
global_predictions.to_netcdf('out/global_predictions.nc')

/var/folders/v7/nxggzv_j5s936v9rvl85gh2w0000gn/T/ipykernel_64293/362052295.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  divs = circle_range.groupby('cluster').apply(circle_ratio)
/var/folders/v7/nxggzv_j5s936v9rvl85gh2w0000gn/T/ipykernel_64293/3095558326.py:25: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  global_predictions = xr.Dataset.from_dataframe(predictions).set_coords(['lon', 'lat']).drop('index')


In [9]:
mounts = predictions.groupby('cluster').mean().reset_index()
kml = simplekml.Kml()
for i, row in mounts.iterrows():
    kml.newpoint(name=f'{row.cluster}', coords=[(row.lon, row.lat, row.z)])
kml.save('out/global_mounts.kml')

In [10]:
! open out/global_mounts.kml